In [ ]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;


-- Create external table 

-- Your code here
CREATE EXTERNAL TABLE posts_sample_external (
    Id string,
    CreationYear string,
    CreationMonth string
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+).*\")(?=.*\\bCreationDate=\"(\\d{4}-\\d+).*\").*' 
    )
LOCATION '/data/stackexchange1000/posts';

In [ ]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here
SELECT * FROM demodb.posts_sample_external
LIMIT 10;

In [ ]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here
-- adding necessary JARs and including database
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
-- ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
    Id string
)
PARTITIONED BY (
    CreationMonth string,
    CreationYear string
);

In [ ]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table

USE demodb;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.enforce.bucketing=true;
SET hive.mapred.supports.subdirectories=true;

-- Your code here for inserting data
FROM posts_sample_external pse
INSERT overwrite table posts_sample
PARTITION (CreationMonth, CreationYear)
SELECT pse.Id, pse.CreationMonth, pse.CreationYear;

In [ ]:
%%writefile task1_watch_new_table.hql
-- Your code here
SELECT * FROM demodb.posts_sample
LIMIT 10;

In [ ]:
%%writefile task1_result.hql
-- Your code here
with cte as (
    select count(*) as total, CreationYear, CreationMonth
    from demodb.posts_sample
    group by CreationYear, CreationMonth
), cte2 as (
    select total, CreationYear, CreationMonth,
        dense_rank() over (order by CreationMonth) as rang
    from cte
)
select CreationYear, CreationMonth, total
from cte2
where rang=3;

In [ ]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

In [ ]:
!cat task1.hql

In [ ]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

In [ ]:
%%bash
hive -f task1.hql